In [1]:
import pandas as pd
from google_play_scraper import reviews, Sort
import warnings

warnings.filterwarnings("ignore")

## 데이터 수집
### 구글 playstore 리뷰 데이터

In [2]:
def gstore_reviews(app_id):
    # 리뷰 가져오기
    result = reviews(
        app_id,
        lang="ko", # 한국어 리뷰
        country="kr", # 한국
        sort=Sort.NEWEST, # 최신 리뷰 먼저 가져오기
        count=100000, # 가져올 리뷰 수 (최대 100,000)
    )

    # 리뷰 데이터
    review_data = result[0]

    # DataFrame으로 변환
    return pd.DataFrame(review_data)

In [3]:
duo_df = gstore_reviews("com.duolingo")
duo_df.to_csv("duo_rv.csv", index=False, encoding="utf-8")

qz_df = gstore_reviews("com.quizlet.quizletandroid")
qz_df.to_csv("qz_rv.csv", index=False, encoding="utf-8")

cc_df = gstore_reviews("classcard.net")
cc_df.to_csv("cc_rv.csv", index=False, encoding="utf-8")

qd_df = gstore_reviews("com.mathpresso.qanda")
qd_df.to_csv("qd_rv.csv", index=False, encoding="utf-8")

In [5]:
# 퀴즈렛
df = pd.read_csv("qz_rv.csv")  # 실제 파일명으로 변경 필요

# 날짜 컬럼을 datetime 형식으로 변환
df["at"] = pd.to_datetime(df["at"])

# 2025년 이후 데이터만 필터링
df_filtered = df[df["at"].dt.year >= 2025]
df_filtered.to_csv("qz_rv_2025.csv", index=False, encoding="utf-8")

### Quizlet의 학습용, 교구용 비율 분석 - 사용자 리뷰 기반

In [4]:
# 학습용 분류 키워드 설정
study_keywords = [
    "시험",
    "문제",
    "단어",
    "개념",
    "복습",
    "암기",
    "테스트",
    "학습",
    "공부",
    "연습",
]

# 교구용 분류 키워드 설정
teaching_keywords = [
    "수업",
    "퀴즈",
    "교사용",
    "학습지",
    "학생",
    "수업자료",
    "활동",
    "플래시카드",
]

def categorize_content(text):
    if isinstance(text, str):  # 문자열 인지 확인
        if any(word in text for word in study_keywords):
            return "학습용"
        elif any(word in text for word in teaching_keywords):
            return "교구용"
    return "기타"

# category 컬럼 추가, values = "학습용", "교구용", "기타"
qz_df["category"] = qz_df["content"].apply(categorize_content)

# 카테고리 별 비율 계산
category_counts = qz_df["category"].value_counts(normalize=True) * 100
category_counts

category
기타     61.069652
학습용    38.163350
교구용     0.766998
Name: proportion, dtype: float64